In [12]:
import ee
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance
    
ee.Authenticate()
ee.Initialize(project="shrubwise-dc-488219")

RAP_veg_yearly_10m = ee.ImageCollection("projects/rap-data-365417/assets/vegetation-cover-10m")

def rap_10m_mosaic_for_year(year: int) -> ee.Image:
    """Return the RAP 10m vegetation-cover mosaic for a year."""
    return RAP_veg_yearly_10m.filter(ee.Filter.eq("year", year)).mosaic()

# Demo AOIs (swap/extend later)
aoi_bliss = ee.Geometry.Rectangle([-120.1018846, 38.99274873, -120.0899834, 39.0020357], geodesic=False)

minx, miny, maxx, maxy = -120.1018846, 38.99274873, -120.0899834, 39.0020357
center_lat = (miny + maxy) / 2
km_shift = 10
dlon = km_shift / (111.32 * np.cos(np.deg2rad(center_lat)))
aoi_demo_shifted = ee.Geometry.Rectangle([minx + dlon, miny, maxx + dlon, maxy], geodesic=False)

AOIS_DEFAULT = {
    "DL_Bliss": aoi_bliss,
    "Demo_Shifted_10km": aoi_demo_shifted,
}

In [20]:
rap_10m_mosaic_for_year(2025).bandNames().getInfo()

['AFG', 'BGR', 'LTR', 'PFG', 'SHR', 'TRE']

In [32]:
def resolve_pft_bands(band_names):
    bn = [b.lower() for b in band_names]
    lookup = {b.lower(): b for b in band_names}

    candidates = {
        "AFG": ["afg", "annual_forb", "annual forb", "annual grass", "annual_forbs", "annual_grasses"],
        "PFG": ["pfg", "perennial_forb", "perennial forb", "perennial grass", "perennial_forbs", "perennial_grasses"],
        "SHR": ["shr", "shrub"],
        "TRE": ["tre", "tree"],
        "LTR": ["ltr", "litter"],
        "BGR":  ["bg", "bare", "bareground", "bare_ground"],
    }

    resolved = {}
    for canon, pats in candidates.items():
        match = None
        for b_lower, b_orig in lookup.items():
            if any(p in b_lower for p in pats):
                match = b_orig
                break
        if match is None:
            return None
        resolved[canon] = match

    return resolved

In [33]:
def qc_range_and_coverage(image, band, aois, scale=10):
    rows = []
    img = image.select(band).toFloat()

    for name, geom in aois.items():
        cnt = img.reduceRegion(ee.Reducer.count(), geom, scale, maxPixels=1e9, bestEffort=True).get(band)
        lo  = img.lt(0).reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e9, bestEffort=True).get(band)
        hi  = img.gt(100).reduceRegion(ee.Reducer.sum(), geom, scale, maxPixels=1e9, bestEffort=True).get(band)

        rows.append({
            "aoi": name,
            "valid_count": ee.Number(cnt).getInfo() if cnt is not None else 0,
            "out_lt0": ee.Number(lo).getInfo() if lo is not None else 0,
            "out_gt100": ee.Number(hi).getInfo() if hi is not None else 0,
        })

    return pd.DataFrame(rows).set_index("aoi")

In [35]:
def qc_distribution_summary(image, band, aois, scale=10):
    img = image.select(band).toFloat()
    reducer = (
        ee.Reducer.mean()
          .combine(ee.Reducer.median(), sharedInputs=True)
          .combine(ee.Reducer.min(), sharedInputs=True)
          .combine(ee.Reducer.max(), sharedInputs=True)
          .combine(ee.Reducer.percentile([5, 25, 75, 95]), sharedInputs=True)
    )

    rows = []
    for name, geom in aois.items():
        stats = img.reduceRegion(reducer, geom, scale, maxPixels=1e9, bestEffort=True).getInfo()

        pct_gt0 = img.gt(0).reduceRegion(
            ee.Reducer.mean(), geom, scale, maxPixels=1e9, bestEffort=True
        ).get(band)

        rows.append({
            "aoi": name,
            "mean": stats.get(f"{band}_mean"),
            "median": stats.get(f"{band}_median"),
            "min": stats.get(f"{band}_min"),
            "max": stats.get(f"{band}_max"),
            "p05": stats.get(f"{band}_p5"),
            "p25": stats.get(f"{band}_p25"),
            "p75": stats.get(f"{band}_p75"),
            "p95": stats.get(f"{band}_p95"),
            "pct_gt0": ee.Number(pct_gt0).getInfo() if pct_gt0 is not None else None,
        })

    return pd.DataFrame(rows).set_index("aoi")

In [36]:
def qc_pft_sum(image, aois, scale=10):
    bands = image.bandNames().getInfo()

    if "BG" in bands:
        bg_band = "BG"
    elif "BGR" in bands:
        bg_band = "BGR"

    pft_bands = ["AFG", "PFG", "SHR", "TRE", "LTR", bg_band]
    pft_sum = image.select(pft_bands).reduce(ee.Reducer.sum()).rename("pft_sum")

    reducer = (
        ee.Reducer.mean()
          .combine(ee.Reducer.min(), sharedInputs=True)
          .combine(ee.Reducer.max(), sharedInputs=True)
    )

    rows = []
    for name, geom in aois.items():
        stats = pft_sum.reduceRegion(reducer, geom, scale, maxPixels=1e9, bestEffort=True).getInfo()
        rows.append({
            "aoi": name,
            "pft_sum_mean": stats.get("pft_sum_mean"),
            "pft_sum_min": stats.get("pft_sum_min"),
            "pft_sum_max": stats.get("pft_sum_max"),
            "bands_used": pft_bands,
        })

    return pd.DataFrame(rows).set_index("aoi")

In [40]:
year = 2025
rap2025 = rap_10m_mosaic_for_year(year)

# Range + coverage
range_cov = qc_range_and_coverage(rap2025, band="SHR", aois=AOIS_DEFAULT, scale=10)
range_cov

,valid_count,out_lt0,out_gt100
aoi,,,
DL_Bliss,13728,0,0
Demo_Shifted_10km,13728,0,0


In [41]:
# Distribution summary
dist = qc_distribution_summary(rap2025, band="SHR", aois=AOIS_DEFAULT, scale=10)
dist

,mean,median,min,max,p05,p25,p75,p95,pct_gt0
aoi,,,,,,,,,
DL_Bliss,2.428642,0,0,24,0,0,4,9.0,0.456482
Demo_Shifted_10km,0.000000,0,0,0,0,0,0,0.0,0.000000


In [42]:
#  PFT sum check (should be close to 100)
pft = qc_pft_sum(rap2025, aois=AOIS_DEFAULT, scale=10)
pft

,pft_sum_mean,pft_sum_min,pft_sum_max,bands_used
aoi,,,,
DL_Bliss,36.542446,0,111,"[AFG, PFG, SHR, TRE, LTR, BGR]"
Demo_Shifted_10km,0.000292,0,1,"[AFG, PFG, SHR, TRE, LTR, BGR]"
